In [1]:
# Importing the libraries necessary for the parsing

import pandas as pd 
import numpy as np
import os
import time
from datetime import datetime
from datetime import date
# from pandas.compat import StringIO
import re

## Global variables

In [2]:
# Column fields for the parser to look for except Title, Release data and Directed By
criteria_list = ['movie_id', 'review_count_user', 'review_count_critic','metascore', 'rating_value', 'rating_count','date_published', 'director_names', 'director_ids']

# Newly added features in an attempt to increase the accuracy
additional_criteria_list = ['duration','Runtime','titleYear','genre_tags',
                            'Country','Language','Budget','Opening Weekend','Gross','Color']

regex_criteria_list = ['title','Release Date']

criteria_list.extend(additional_criteria_list)
criteria_list.extend(regex_criteria_list)

In [3]:
len(criteria_list)

21

In [4]:
# Location of all the 700K+ files (703243)

# workingDir = 'C:\\Users\\VictorY\\Desktop\\TestData\\sample'

# #Sample Directory
# workingDir = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\Sample'
# folder_path = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\Sample'

# Main Directory
workingDir = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\movie_info\\movie_bios'
folder_path = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\movie_info\\movie_bios'



# Functions

In [5]:
def exact_search_word(search,line):
    result = re.findall('\\b'+search+'\\t', line, flags=re.IGNORECASE)
    if len(result) == 1:
        return True
    else:
        return False

In [6]:
# Function: Helper method fo the parse_file function
# Reads line by line and looks for the criterion list in the first n letters and separates the tab and new line characters
# n corresponds to the lenght of the criteria at that particular loop

def process_criteria_list(line, collection): #helper method for the Parse_File function
    for criterion in criteria_list:
        if exact_search_word(criterion,line[:len(criterion)+1]):
            line.split('\n')
            collection.append(line.strip().split('\t',1))

In [7]:
# # Function: Helper method fo the parse_file function
# # Reads line by line and looks for the criterion list and separates the tab and new line characters

# def process_criteria_list(line, collection): #helper method for the Parse_File function
#     for criterion in criteria_list:
#         if criterion in line[:20]:
#             line.split('\n')
#             collection.append(line.strip().split('\t',1))

In [8]:
# Function: Transpose the values to columns and fetch the column with the values alone

def modify_data(list): #Helper method Parse_File function
    modified_list = np.array(list).T
    return modified_list[1]


In [9]:
# Function: Parse each file with the defined criteria list columns and regex

def parse_file(nameFile,file_location):  
    extract_data = []
    with open(file_location+'/'+nameFile, encoding="utf8") as f:
        for line in f:
            process_criteria_list(line, extract_data)
#             # regex to match the TITLE field
#             regexTitle = r"\btitle\t" 
#             if re.match(regexTitle, line):
#                 extract_data.append(line.strip().split('\t'))
            
#             # regex to match the Release Date
#             regexReleaseDate = r"^.*?\bRelease Date\b.*?$"
#             if re.match(regexReleaseDate,line[:20]):
#                 #check1 = line.strip().split('\t',1)
#                 extract_data.append(line.strip().split('\t'))
            
            # regex to match the Directed by field
            regexDirectedBy = r"^\bDirected by\b.*?$"
            if re.match(regexDirectedBy,line[:20]):
                check = line.strip().split('\t',1)
            # Check if value exists    
                if len(check) == 1:
                    check.append('None')
                    extract_data.append(check)
                else:
                    extract_data.append(check)
                    
#             # regex to match the Country
#             regexCountry = r"^.*?\bCountry\b.*?$"
#             if re.match(regexCountry,line[:10]):
#                 #check1 = line.strip().split('\t',1)
#                 extract_data.append(line.strip().split('\t'))

#             # regex to match the Color
#             regexColor = r"^.*?\bColor\b.*?$"
#             if re.match(regexColor,line[:10]):
#                 #check1 = line.strip().split('\t',1)
#                 extract_data.append(line.strip().split('\t'))
                
#             # regex to match the Gross
#             regexGross = r"^.*?\bGross\b.*?$"
#             if re.match(regexGross,line[:10]):
#                 #check1 = line.strip().split('\t',1)
#                 extract_data.append(line.strip().split('\t')) 
                
#     print("Extracted Data:\n\n",extract_data)
#     print("Modified data:\n\n",modify_data(extract_data))
    return modify_data(extract_data)

In [10]:
# Helper method for save_to_file

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [11]:
# Function: Save Dataframe to CSV

def save_to_file(dataFrame, folderPath):
    today = date.today()
    todayDate = today.strftime("%b-%d-%Y")
    dataFrame.to_csv(folderPath+ '\\' + get_df_name(dataFrame) + '_' + todayDate  + '.csv',index=False, encoding="utf8" )

# Initiate the parsing of files

In [12]:
start = time.time()
print("Started Reading files at :" + str(datetime.utcfromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S')))

final_list = [] #list where all data from all files will be added before converted to dataframe
error_list = []#to keep track of error files 

listed_allFiles = os.listdir(workingDir)
#fileCount = 0    
for file in listed_allFiles:
    filename = os.fsdecode(file)
    if filename.endswith(".dat"):
        try:
            # print(os.path.join(directory, filename))
            single_file_data = parse_file(file,workingDir)
            final_list.append(single_file_data)
        except:
            error_list.append(filename)
    
raw_parsed_data = pd.DataFrame(list(map(np.ravel, final_list)))
errorList = pd.DataFrame(error_list)

# Stop timer

end = time.time()
print("Stopped Reading files at :" + str(datetime.utcfromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')))
print("Total time taken in minutes: " + str((end - start)/60))

Started Reading files at :2020-04-11 12:06:36
Stopped Reading files at :2020-04-11 13:32:29
Total time taken in minutes: 85.8772578159968


Approximately 90 minutes

In [13]:
safekeepingRaw = raw_parsed_data.copy()
safekeepingRaw

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,nm0005690,USA,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,None,Color(hand painted),nm0721526
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,nm0005690,USA,None,9 May 1893 (USA) See more »,None,None,None,1 min,Black and White,nm0005690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699765,tt6751234,T.Y (2016) - IMDb,None,None,None,9.6,5,2016-01-01,None,2016,...,nm8906630,India,Hindi,1 January 2016 (India) See more »,"INR 10,000 (estimated)",None,None,None,Color(HD),nm8906630
699766,tt6753894,La Flor: Primera Parte (2016) - IMDb,None,None,None,7.8,9,2016-11-23,3h40min,2016,...,nm1183471,Argentina,Spanish,23 November 2016 (Argentina) See more »,None,None,None,220 min,Color,nm1183471
699767,tt6756658,The Mystery of Godliness (2016) - IMDb,1,None,None,8.8,8,2016-05-12,3h,2016,...,None,South Korea,English,12 May 2016 (South Korea) See more »,"$200,000 (estimated)",None,None,180 min (internet),Color,None
699768,tt6768554,A Virgin Story (2016) - IMDb,None,None,None,9.7,14,2016-11-12,13min,2016,...,nm8916504,India,Bengali,12 November 2016 (India) See more »,"INR 1,000 (estimated)",None,None,13 min,Color,nm8916504


In [14]:
# # In case you get 13 columns, because of incorrect regex expressions

# corruptedrecords = raw_parsed_data.loc[~raw_parsed_data[12].isna()]
# # raw_parsed_data.drop(corruptedrecords.index, inplace= True)

# # Allocating column names to dataframe 

# # raw_parsed_data.drop(raw_parsed_data.columns[12], axis = 1, inplace = True)

In [15]:
# Column header list to fill for the parsed dataframe
title_list = ['movie_id','title','review_count_user','review_count_critic','metascore',
              'rating_value','rating_count','date_published','duration','title_year',
              'genre_tags','director_names','director_ids','Country','Language',
              'release_date','Budget','opening_weekend','Gross','Runtime',
              'Color','directed_by']

# Allocating column names to dataframe 
raw_parsed_data.columns = title_list
raw_parsed_data.head(5)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,director_ids,Country,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,nm0005690,USA,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,None,Color(hand painted),nm0721526
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,nm0005690,USA,None,9 May 1893 (USA) See more »,None,None,None,1 min,Black and White,nm0005690


In [16]:
# Intermediate save as raw data after parsing
save_to_file(raw_parsed_data,folder_path)

# Error Files
save_to_file(errorList,folder_path)
print("Length of the error list:" + str(len(error_list)))

Length of the error list:3473


In [17]:
# Copy for safe keeping before preprocessing the data
preProcessingDF = raw_parsed_data.copy()

### Start point : After parsing

In [107]:
raw_parsed_data_fileLocation = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Data\\IMDb\\raw_parsed_data_Apr-11-2020.csv'

In [108]:
preProcessingDF = pd.read_csv(raw_parsed_data_fileLocation,encoding="utf8")

In [109]:
print("\nDataframe Shape of parsed data : ",str(preProcessingDF.shape))


Dataframe Shape of parsed data :  (699770, 22)


# Preprocessing the data

In [110]:
preProcessingDF.head(3)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,director_ids,Country,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,nm0005690,USA,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,nm0721526,France,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526


### Generate two new columns : Release Year & Movie Year

In [111]:
# Date published column recorded incorrect dates for most records
# Record the dates of when the movies were loaded into the IMDB and not the release date

# Resolution : Extract the years from release date column and movie column

preProcessingDF['release_year'] = preProcessingDF.release_date.str.extract('(\d{4})', expand=False).str.strip()
preProcessingDF['movie_year'] = preProcessingDF.title.str.extract('\([^\d]*(\d{4})[^\d]*\)(?=[^()]*$)', expand=False).str.strip()

In [112]:
preProcessingDF.head(5)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690,1894,1894
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526,1892,1892
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526,1892,1892
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,None,28 October 1892 (France) See more »,None,None,None,None,Color(hand painted),nm0721526,1892,1892
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,None,9 May 1893 (USA) See more »,None,None,None,1 min,Black and White,nm0005690,1893,1893


### Remove and replace undesired entries

In [113]:
# Replace the movie_id which has .zip in their id
preProcessingDF['movie_id'] = preProcessingDF['movie_id'].str.replace('.zip','')

# There are few movie directors which have tab in their AKA names. Extract only the names with this regex replacer
preProcessingDF['director_names'] = preProcessingDF.director_names.str.replace('\(([^)]+)\)','')

In [114]:
# Movie records that doesn't contain proper format of data
print("Count of records which doesn not have the proper format is : ", len(preProcessingDF[~preProcessingDF['movie_id'].str.contains('tt')]))
preProcessingDF[~preProcessingDF['movie_id'].str.contains('tt')]


Count of records which doesn not have the proper format is :  5780


,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
2477,nm0826094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14209,nm0849806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15868,nm0823864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28194,nm0455839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35250,nm0404014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698843,nm2712890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698945,nm4024927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
698982,nm0264269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699104,nm2670218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# Remove movie_id without proper format

preProcessingDF = preProcessingDF[preProcessingDF['movie_id'].str.contains('tt')]
preProcessingDF.shape

(693990, 24)

In [116]:
# The number of records which doesn't contain a year after processing
print(" Before Removing : Number of records left without year : ",len(preProcessingDF.loc[preProcessingDF['movie_year'].isna(),'movie_year']))

 Before Removing : Number of records left without year :  10544


#### Approach where we keep main reference as movie_year and copy from release_year followed by published_year

In [117]:
# Approach where we keep main reference as movie_year and copy from release_year followed by published_year

# Fill missing values in movie_year with that of release_yearyear
preProcessingDF.loc[preProcessingDF['movie_year'].isna(),'movie_year'] = preProcessingDF['release_year']

# The number of records which doesn't contain a year after processing
print(" Before Removing : Number of records left without year :",len(preProcessingDF.loc[preProcessingDF['movie_year'].isna(),'movie_year']))

# Cross check if any movie_year value is missing further 
preProcessingDF = preProcessingDF.loc[~preProcessingDF['movie_year'].isna()]
print(" After Removing : Number of records left without year :",len(preProcessingDF.loc[preProcessingDF['movie_year'].isna(),'movie_year']))

# Convert the type of movie year to integer for calculations
preProcessingDF.movie_year = preProcessingDF.movie_year.astype(int)

 Before Removing : Number of records left without year : 716
 After Removing : Number of records left without year : 0


In [118]:
print("\nDataframe Shape at this step : ",str(preProcessingDF.shape))


Dataframe Shape at this step :  (693274, 24)


In [119]:
preProcessingDF.head()

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690,1894,1894
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526,1892,1892
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526,1892,1892
3,tt0000004,Egy jó pohár sör (1892) - IMDb,1,None,None,6.6,90,1892-10-28,None,1892,...,None,28 October 1892 (France) See more »,None,None,None,None,Color(hand painted),nm0721526,1892,1892
4,tt0000005,A patkolókovács (1893) - IMDb,18,3,None,6.2,1499,1893-05-09,1min,1893,...,None,9 May 1893 (USA) See more »,None,None,None,1 min,Black and White,nm0005690,1893,1893


In [120]:
# Consider the published year column if required

#preProcessingDF.loc[preProcessingDF['movie_year'].isna(),'movie_year'] = preProcessingDF['published_year']
#preProcessingDF.movie_year.value_counts(dropna = False)

## Fill Missing Director ID info from IMDB Crew dataset and directed_by column

In [121]:
print("The number of movies containing None in the director ID column",len(preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False))]))

The number of movies containing None in the director ID column 127803


In [122]:
preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False))].head()

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
37,tt0000039,Barnet Horse Fair (1896) - IMDb,None,None,None,5.1,10,None,None,1896,...,None,None,None,None,None,None,Black and White,NaN,NaN,1896
102,tt0000104,Pompiers à Lyon (1896) - IMDb,6,1,None,5.6,302,2000-10-11,1min,1896,...,None,None,None,None,None,1 min,Black and White,NaN,NaN,1896
131,tt0000134,The Wandering Negro Minstrels (1896) - IMDb,None,None,None,7.0,5,None,None,1896,...,None,February 1896 (UK) See more »,None,None,None,None,Black and White,NaN,1896,1896
153,tt0000162,A Niagara-vízesés (1897) - IMDb,8,1,None,5.6,313,2000-10-11,1min,1897,...,None,None,None,None,None,1 min,Black and White,NaN,NaN,1897
187,tt0000225,La belle et la bête (1899) - IMDb,None,None,None,6.0,12,None,None,1899,...,None,None,None,None,None,None,Black and White,NaN,NaN,1899


### Fill missing director_ids from crew dataset

In [123]:
IMDBCrewDatafileLocation = 'C:\\Yuva\\ITU\\3rd Sem\\Research Topics\\Movies & Gender\\Website - IMDB\\title\\data.tsv'

In [124]:
IMDBCrewDF = pd.read_csv(IMDBCrewDatafileLocation, sep='\t')
print(IMDBCrewDF.shape)

(6710061, 3)


In [125]:
# Drop writers column and rename tconst to movie_id
IMDBCrewDF.drop(columns = 'writers',inplace=True)
IMDBCrewDF.rename(columns={'tconst':'movie_id','directors':'IMDB_director_id'}, inplace=True)

In [126]:
# Replace all /N values directods as None

IMDBCrewDF.loc[(IMDBCrewDF['IMDB_director_id'].str.contains('N',na=False)), 'IMDB_director_id'] = 'None'
IMDBCrewDF['IMDB_director_id'].fillna('None',inplace=True)
IMDBCrewDF.dropna(inplace=True)

print(IMDBCrewDF.shape)

(6710061, 2)


In [127]:
# IMDB Crew had a different format of data for movies with multiple directors. 
# Convert it to the format we want, i.e '\t'
IMDBCrewDF.replace({',': '\t'}, regex=True,inplace= True)

#### Merge the IMDBCrew data with our pre processing data to fill the missing director_ids

In [128]:
# Merge the IMDBCrew data with our pre processing data to fill the missing director_ids
print("\n preProcessingDF Dataframe Shape Before merge : ",str(preProcessingDF.shape))
preProcessingDF = preProcessingDF.merge(IMDBCrewDF, on='movie_id', how ='left')
print("\n preProcessingDF dataframe Shape After merge : ",str(preProcessingDF.shape))


 preProcessingDF Dataframe Shape Before merge :  (693274, 24)

 preProcessingDF dataframe Shape After merge :  (693274, 25)


In [129]:
preProcessingDF['IMDB_director_id'].isna().sum()

1452

In [130]:
preProcessingDF['IMDB_director_id'].fillna('None',inplace=True)

In [131]:
preProcessingDF['IMDB_director_id'].isna().sum()

0

In [132]:
prior_fill_with_IMDB_director_id = len(preProcessingDF[preProcessingDF['director_ids'].str.contains('None')])

In [133]:
preProcessingDF[preProcessingDF['director_ids'].str.contains('None')][['movie_id','director_ids','IMDB_director_id']]

,movie_id,director_ids,IMDB_director_id
37,tt0000039,None,None
102,tt0000104,None,None
131,tt0000134,None,None
153,tt0000162,None,nm0698645
187,tt0000225,None,None
...,...,...,...
693253,tt6705808,None,nm8881418\tnm8881417
693265,tt6725158,None,nm0000005
693267,tt6729316,None,None
693271,tt6756658,None,None


In [134]:
# Replace all the missing director IDs  from the directors column of Original IMDB data tsv
preProcessingDF['IMDB_director_id'].fillna('None',inplace=True)
preProcessingDF.loc[preProcessingDF['director_ids'].str.contains('None'),'director_ids'] = preProcessingDF['IMDB_director_id']

In [135]:
post_fill_with_IMDB_director_id = len(preProcessingDF[preProcessingDF['director_ids'].str.contains('None')])

In [136]:
preProcessingDF[preProcessingDF['director_ids'].str.contains('None')][['movie_id','director_ids','IMDB_director_id']]

,movie_id,director_ids,IMDB_director_id
37,tt0000039,None,None
102,tt0000104,None,None
131,tt0000134,None,None
187,tt0000225,None,None
189,tt0000229,None,None
...,...,...,...
693246,tt6695448,None,None
693247,tt6695458,None,None
693267,tt6729316,None,None
693271,tt6756658,None,None


In [137]:
print('Records filled with director ID by referencing IMDB crew is : ', prior_fill_with_IMDB_director_id - post_fill_with_IMDB_director_id)

Records filled with director ID by referencing IMDB crew is :  56440


In [139]:
# Drop the IMDB Director ID column as it is no longer relevant

preProcessingDF.drop(columns= 'IMDB_director_id',inplace= True )

In [140]:
print("\n preProcessingDF dataframe Shape After merge with IMDB Crew data : ",str(preProcessingDF.shape))


 preProcessingDF dataframe Shape After merge with IMDB Crew data :  (693274, 24)


### Fill director_ids from the directed by column

In [141]:
# Number of records with no director names or director IDs

#preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False) | preProcessingDF.director_ids.isnull())].count()
print("Number of records with no director names or director IDs:",
      len(preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False))]))

Number of records with no director names or director IDs: 71363


In [142]:
# Number of records with no director names or director IDs

#preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False) | preProcessingDF.director_ids.isnull())].count()
print("Number of records with no director names or director IDs:",
      len(preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False))]))

Number of records with no director names or director IDs: 71363


In [143]:
# To strip spaces in the directed by column before copying this information to the missing director information
preProcessingDF.directed_by = preProcessingDF.directed_by.str.strip()
preProcessingDF.director_ids = preProcessingDF.director_ids.str.strip()

# To copy Directed by to director_ids if the value is None in director_ids 
# ( 71363 - 71331 records were corrected with missing director info)
preProcessingDF.loc[preProcessingDF['director_ids'] == "None",'director_ids'] = preProcessingDF['directed_by']

## Alternatively
#preProcessingDF.director_ids = preProcessingDF.Directed_by.where(preProcessingDF.director_ids == 'None', preProcessingDF.director_ids)

In [144]:

# Number of records with no director names or director IDs

preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False) | preProcessingDF.director_ids.isnull())].count()

movie_id               71331
title                  71331
review_count_user      71331
review_count_critic    71331
metascore              71331
rating_value           71331
rating_count           71331
date_published         71331
duration               71331
title_year             71331
genre_tags             71331
director_names         71331
director_ids               0
Country                71331
Language               71331
release_date           71331
Budget                 71331
opening_weekend        71331
Gross                  71331
Runtime                71331
Color                  71331
directed_by                0
release_year           67053
movie_year             71331
dtype: int64

In [147]:
preProcessingDF[(preProcessingDF.director_ids.str.contains("None", na=False) | preProcessingDF.director_ids.isnull())].head()

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
37,tt0000039,Barnet Horse Fair (1896) - IMDb,None,None,None,5.1,10,None,None,1896,...,None,None,None,None,None,None,Black and White,NaN,NaN,1896
102,tt0000104,Pompiers à Lyon (1896) - IMDb,6,1,None,5.6,302,2000-10-11,1min,1896,...,None,None,None,None,None,1 min,Black and White,NaN,NaN,1896
131,tt0000134,The Wandering Negro Minstrels (1896) - IMDb,None,None,None,7.0,5,None,None,1896,...,None,February 1896 (UK) See more »,None,None,None,None,Black and White,NaN,1896,1896
187,tt0000225,La belle et la bête (1899) - IMDb,None,None,None,6.0,12,None,None,1899,...,None,None,None,None,None,None,Black and White,NaN,NaN,1899
189,tt0000229,Casey at the Bat (1899) - IMDb,2,None,None,4.7,53,2007-10-18,1min,1899,...,None,April 1899 (USA) See more »,None,None,None,None,Black and White,NaN,1899,1899


In [149]:
# Remove the director_ids with None and null

stage1PreProcessedDF = preProcessingDF[~(preProcessingDF.director_ids.str.contains("None", na=False) | preProcessingDF.director_ids.isnull())]

# Remove the director_ids with null
#stage1PreProcessedDF = stage1PreProcessedDF[~stage1PreProcessedDF.director_ids.isnull()]

In [150]:
stage1PreProcessedDF.head(3)

,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,date_published,duration,title_year,...,Language,release_date,Budget,opening_weekend,Gross,Runtime,Color,directed_by,release_year,movie_year
0,tt0000001,Carmencita - spanyol tánc (1894) - IMDb,12,2,None,5.8,1255,1894-03-10,1min,1894,...,None,10 March 1894 (USA) See more »,None,None,None,1 min,Black and White,nm0005690,1894,1894
1,tt0000002,A bohóc és kutyái (1892) - IMDb,0,None,None,6.5,148,1892-10-28,5min,1892,...,None,28 October 1892 (France) See more »,None,None,None,5 min,Color(hand painted),nm0721526,1892,1892
2,tt0000003,Szegény Pierrot (1892) - IMDb,12,5,None,6.6,819,1892-10-28,4min,1892,...,None,28 October 1892 (France) See more »,None,None,None,4 min (1996 reconstitution),Color(hand painted),nm0721526,1892,1892


In [151]:
print("\nDataframe Shape of partially cleaned data : ",str(stage1PreProcessedDF.shape))


Dataframe Shape of partially cleaned data :  (621943, 24)


In [152]:
stage1PreProcessedDF.columns.values

array(['movie_id', 'title', 'review_count_user', 'review_count_critic',
       'metascore', 'rating_value', 'rating_count', 'date_published',
       'duration', 'title_year', 'genre_tags', 'director_names',
       'director_ids', 'Country', 'Language', 'release_date', 'Budget',
       'opening_weekend', 'Gross', 'Runtime', 'Color', 'directed_by',
       'release_year', 'movie_year'], dtype=object)

In [153]:
# Save files to desired folder path

save_to_file(stage1PreProcessedDF,folder_path)

# End of Parser and partial pre-processing